In [29]:
import pandas as pd
import numpy as np
import seaborn as sns

In [30]:
df=pd.read_csv('data/finalTrain.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [31]:
df.shape

(45584, 20)

In [32]:
df.drop(columns=['Order_Date'],axis=1,inplace=True)

In [33]:
from pandas_profiling import ProfileReport
pf = ProfileReport(df)

In [34]:
# pf.to_widgets()

In [35]:
#As we can notice high correlation between (Delivery_location_latitude,Restaurant_latitude) and (Restaurant_longitude,Delivery_location_longitude) so we can remove each feature from both of the groups
df.drop(columns=['Restaurant_latitude','Restaurant_longitude'],inplace=True)

In [36]:
#Dropping id column
df.drop(columns=['ID'],inplace=True)

In [37]:
df.isnull().sum()

Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Delivery_location_latitude        0
Delivery_location_longitude       0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [38]:
df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].median(),inplace=True)
df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].mode()[0],inplace=True)
df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0],inplace=True)

In [39]:
df.select_dtypes('number').isna().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Delivery_location_latitude     0
Delivery_location_longitude    0
Vehicle_condition              0
multiple_deliveries            0
Time_taken (min)               0
dtype: int64

In [40]:
df.select_dtypes('object').isna().sum()

Delivery_person_ID         0
Time_Orderd             1731
Time_Order_picked          0
Weather_conditions       616
Road_traffic_density     601
Type_of_order              0
Type_of_vehicle            0
Festival                 228
City                    1200
dtype: int64

In [41]:
df['City'].fillna(df['City'].mode()[0],inplace=True)
df['Festival'].fillna(df['Festival'].mode()[0],inplace=True)
df['Road_traffic_density'].fillna(df['Road_traffic_density'].mode()[0],inplace=True)
df['Time_Orderd'].fillna(df['Time_Orderd'].mode()[0],inplace=True)
df['Weather_conditions'].fillna(df['Weather_conditions'].mode()[0],inplace=True)

In [42]:
df.select_dtypes('object').isna().sum()

Delivery_person_ID      0
Time_Orderd             0
Time_Order_picked       0
Weather_conditions      0
Road_traffic_density    0
Type_of_order           0
Type_of_vehicle         0
Festival                0
City                    0
dtype: int64

In [43]:
df.isnull().sum()

Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Delivery_location_latitude     0
Delivery_location_longitude    0
Time_Orderd                    0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
dtype: int64

In [ ]:
# Tommorow's task 
# convert time to numeric data 

In [44]:
#Segregate numerical columns and categorical columns
# numerical_columns = df.columns.where(df.dtypes!='o')
# categorical_columns = df.columns[df.dtypes=='object']
# print('numerical_columns',numerical_columns)
# print('categorical_columns',categorical_columns)

numerical_columns Index(['Delivery_person_ID', 'Delivery_person_Age', 'Delivery_person_Ratings',
       'Delivery_location_latitude', 'Delivery_location_longitude',
       'Time_Orderd', 'Time_Order_picked', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City',
       'Time_taken (min)'],
      dtype='object')
categorical_columns Index(['Delivery_person_ID', 'Time_Orderd', 'Time_Order_picked',
       'Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')


In [45]:
# df[categorical_columns]

,Delivery_person_ID,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City
0,DEHRES17DEL01,21:55,22:10,Fog,Jam,Snack,motorcycle,No,Metropolitian
1,KOCRES16DEL01,14:55,15:05,Stormy,High,Meal,motorcycle,No,Metropolitian
2,PUNERES13DEL03,17:30,17:40,Sandstorms,Medium,Drinks,scooter,No,Metropolitian
3,LUDHRES15DEL02,09:20,09:30,Sandstorms,Low,Buffet,motorcycle,No,Metropolitian
4,KNPRES14DEL02,19:50,20:05,Fog,Jam,Snack,scooter,No,Metropolitian
...,...,...,...,...,...,...,...,...,...
45579,JAPRES04DEL01,11:35,11:45,Windy,High,Meal,motorcycle,No,Metropolitian
45580,AGRRES16DEL01,19:55,20:10,Windy,Jam,Buffet,motorcycle,No,Metropolitian
45581,CHENRES08DEL03,23:50,24:05:00,Cloudy,Low,Drinks,scooter,No,Metropolitian
45582,COIMBRES11DEL01,13:35,13:40,Cloudy,High,Snack,motorcycle,No,Metropolitian
